**Problem statement:** 

Continuous bag of words (cbow) word2vec word embedding work is that it tends to predict the probability of a word given a context. A context may be a single word or a group of words. But for simplicity, I will take a single context word and try to predict a single target word. 

The purpose of this assignment is to be able to create a word embedding for the  given data set.  

**Data set :** w2v.txt 

### Importing libraries

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense
from scipy import spatial

### Function Definations

In [2]:
# Finding the maximum length of an array required to include all sentences. Vocabulary size, in order to create the 
# same size inputs by padding with zeros

def get_max_vocabulary_size(encoded_line_words):
    max_voc_size = 0
    max_integer_index = 0
    for l in encoded_line_words:
        if (len(l) > 0) and (np.argmax(l) > max_integer_index):
            max_integer_index = l[np.argmax(l)]
        if max_voc_size < len(l):
            max_voc_size = len(l)
    return max_voc_size

In [3]:
# Creating a new list with the targets for the words with window size as window_size. 
# Only positive window size is taken into consideration

def map_target(context, window_size):
    if window_size < 0:
        print("Only Positive window sie is expected.")
        return;
    target = context[window_size:]
    m = 10
    i = 0
    while i < window_size :
        target = np.append(target, [0])
        i = i + 1
    return np.array(target)

In [4]:
# Creating array of targets using the map_target function

def map_all_targets(context_list, window_size):
    targets = []
    for context in context_list:
        target_row = map_target(context, window_size)
        targets.append(np.array(target_row))
    return targets

In [5]:
def pad_sequences_words(file_lines_words, maxlen):
    padded_word = []
    for line_word in file_lines_words:
        blank_word_index = len(line_word)
        while blank_word_index < maxlen:
            line_word.append("")
            padded_word.append(line_word)
            blank_word_index = blank_word_index + 1

In [6]:
import re
def corpus_to_vocab(txt_file):
    corpus = ''
    with open(txt_file, 'r', encoding="utf-8") as file:
        try:
            for line in file:
                try:
                    line = line.strip()
                    if line!="":
                        corpus = corpus + ' ' + line
                except Exception as E:
                    print ("got An exception 2: ", E)
                    pass         
        except Exception as E:
            print ("got An exception 3: ", E)
            pass
        corpus = re.sub('[\.\,\"\'\(\)\n\s]+', ' ', corpus.strip().lower())
        return corpus.split(' ')

In [7]:
def unique_words_list(word_list):
    unique_words = []
    for w in word_list:
        if (w not in unique_words) and len(w)>2:
            unique_words.append(w)
    return unique_words

In [8]:
def word_OHE(vocabulary):
    OHE_word = [one_hot(wd.strip(),len(vocabulary))[0] for wd in vocabulary]
    OHE_word_pair = {wd.strip():one_hot(wd.strip(),len(vocabulary))[0] for wd in vocabulary}
    Word_OHE_pair = {one_hot(wd.strip(),len(vocabulary))[0]:wd.strip() for wd in vocabulary}
    return {'Word':vocabulary, 'OHE': OHE_word, 'Word_OHE':OHE_word_pair, 'OHE_Word': Word_OHE_pair}

In [9]:
w_list = corpus_to_vocab("w2v.txt")

In [10]:
vocabulary = unique_words_list(w_list)

In [11]:
dictionary = word_OHE(vocabulary)

In [12]:
targets = [target for target in dictionary['OHE']]

In [13]:
dictionary['OHE'][0]

26

In [14]:
index = 0
targets = []
for target in dictionary['OHE']:
    if index < len(dictionary['OHE'])-1:
        targets.append(dictionary['OHE'][index+1])
    index = index + 1
targets.append(0)

In [15]:
len(vocabulary)

255

In [16]:
len(targets)

255

In [17]:
context = np.array(dictionary['OHE'])

In [18]:
targets = np.array(targets)

In [19]:
# All the parameters calculation and constants are being set using this function. 
# Any changes to the parameters should be mafe here to avoid execution errors.

def parameters(context):
    reshaped_context = context.reshape(-1)
    INPUT_DIM = max(reshaped_context)+1
    OUTPUT_DIM = 8
    INPUT_LENGTH = len(context)
    EPOCHS = 1000
    VERBOSE = 1
    LOSS = 'categorical_crossentropy'
    ACTIVATION = 'softmax'
    OPTIMIZER = 'adam'
    MATRIX = ['accuracy']
    BESTMODEL = 'embeddings.h5'
    return (INPUT_DIM, OUTPUT_DIM, INPUT_LENGTH, EPOCHS, VERBOSE, LOSS, ACTIVATION, OPTIMIZER, MATRIX, BESTMODEL)

In [20]:
context

array([ 26, 141, 200, 231, 100, 195, 106,  89,   4,  46,  43, 204, 177,
       238,  77, 187, 227,  88, 112, 149, 163, 136,  46, 250,  27, 233,
       242, 186, 180,  24, 228,  24, 186, 165,  33,   5,  40, 202, 106,
       114, 213,  35, 228, 220,  27, 138,  92,  83,  32,  65, 122, 198,
        87, 171, 226, 243,  79, 248, 173,  71, 239, 123,  52, 245, 227,
       193, 191, 223, 128, 124, 190,  81,  63,   2, 143, 209, 233, 152,
       128,  19,  76, 145, 117, 194, 145,  32, 145,  96, 157,  98, 252,
        23,   5,  54,  54, 144, 215,  66,  73, 227,  46,  23, 113, 196,
       184, 139,  84, 227,  53, 106, 222, 132, 142,  28, 210,  99,  84,
        53, 202,  76, 167,  17,  71, 207, 152, 227, 161, 209, 152, 248,
       134, 142, 198, 107,  63, 240, 148, 230,  14, 196,  18,  88, 233,
        86, 218,  11, 224,  58,  34, 105, 138,  89, 156,  64, 233,  20,
       109,  72,  35, 110, 105,  67, 200, 106, 211,   7,  81, 192, 146,
       160,  16, 164,  50, 207, 103,  20, 207, 101, 178, 239,  1

In [21]:
# Confirming that the shapes of context and targets are same

context.shape, targets.shape

((255,), (255,))

In [22]:
context = context.reshape(-1, 255)
targets = targets.reshape(-1, 255)

In [23]:
context.shape, targets.shape

((1, 255), (1, 255))

In [37]:
# Unit testing the context and it's relavent target. Change the index between 0 to 12 to check the pair

index = 0
context[index], targets[index]

(array([ 26, 141, 200, 231, 100, 195, 106,  89,   4,  46,  43, 204, 177,
        238,  77, 187, 227,  88, 112, 149, 163, 136,  46, 250,  27, 233,
        242, 186, 180,  24, 228,  24, 186, 165,  33,   5,  40, 202, 106,
        114, 213,  35, 228, 220,  27, 138,  92,  83,  32,  65, 122, 198,
         87, 171, 226, 243,  79, 248, 173,  71, 239, 123,  52, 245, 227,
        193, 191, 223, 128, 124, 190,  81,  63,   2, 143, 209, 233, 152,
        128,  19,  76, 145, 117, 194, 145,  32, 145,  96, 157,  98, 252,
         23,   5,  54,  54, 144, 215,  66,  73, 227,  46,  23, 113, 196,
        184, 139,  84, 227,  53, 106, 222, 132, 142,  28, 210,  99,  84,
         53, 202,  76, 167,  17,  71, 207, 152, 227, 161, 209, 152, 248,
        134, 142, 198, 107,  63, 240, 148, 230,  14, 196,  18,  88, 233,
         86, 218,  11, 224,  58,  34, 105, 138,  89, 156,  64, 233,  20,
        109,  72,  35, 110, 105,  67, 200, 106, 211,   7,  81, 192, 146,
        160,  16, 164,  50, 207, 103,  20, 207, 101

In [25]:
INPUT_DIM, OUTPUT_DIM, INPUT_LENGTH, EPOCHS, VERBOSE, LOSS, ACTIVATION, OPTIMIZER, MATRIX, BESTMODEL = \
parameters(context)

In [26]:
# Calback array to save the best model. Best model is decided on the basis of accuracy.

callback_list = [tf.keras.callbacks.ModelCheckpoint(filepath=BESTMODEL, monitor = MATRIX[0], 
                                                            save_best_only=True)]

<IPython.core.display.Javascript object>

In [27]:
INPUT_DIM

253

In [28]:
# Creating the sequential model with Embedding layer with output dimention of 8
# The weights of the first layers will be used as the word embedding
# these weights will later be used to calculate the distance between the words.
# least distant words are most related and vice versa

model = Sequential()
embedding_layer = Embedding(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM, input_length=255)
model.add(embedding_layer)
model.add(Flatten())

# A softmax activation is used.

model.add(Dense(255, activation = ACTIVATION))

model.compile(optimizer = OPTIMIZER, loss=LOSS, metrics = MATRIX)

2022-04-04 18:23:26.524892: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
# Summary of the model 

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 255, 8)            2024      
_________________________________________________________________
flatten (Flatten)            (None, 2040)              0         
_________________________________________________________________
dense (Dense)                (None, 255)               520455    
Total params: 522,479
Trainable params: 522,479
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
context

array([[ 26, 141, 200, 231, 100, 195, 106,  89,   4,  46,  43, 204, 177,
        238,  77, 187, 227,  88, 112, 149, 163, 136,  46, 250,  27, 233,
        242, 186, 180,  24, 228,  24, 186, 165,  33,   5,  40, 202, 106,
        114, 213,  35, 228, 220,  27, 138,  92,  83,  32,  65, 122, 198,
         87, 171, 226, 243,  79, 248, 173,  71, 239, 123,  52, 245, 227,
        193, 191, 223, 128, 124, 190,  81,  63,   2, 143, 209, 233, 152,
        128,  19,  76, 145, 117, 194, 145,  32, 145,  96, 157,  98, 252,
         23,   5,  54,  54, 144, 215,  66,  73, 227,  46,  23, 113, 196,
        184, 139,  84, 227,  53, 106, 222, 132, 142,  28, 210,  99,  84,
         53, 202,  76, 167,  17,  71, 207, 152, 227, 161, 209, 152, 248,
        134, 142, 198, 107,  63, 240, 148, 230,  14, 196,  18,  88, 233,
         86, 218,  11, 224,  58,  34, 105, 138,  89, 156,  64, 233,  20,
        109,  72,  35, 110, 105,  67, 200, 106, 211,   7,  81, 192, 146,
        160,  16, 164,  50, 207, 103,  20, 207, 101

In [31]:
targets

array([[141, 200, 231, 100, 195, 106,  89,   4,  46,  43, 204, 177, 238,
         77, 187, 227,  88, 112, 149, 163, 136,  46, 250,  27, 233, 242,
        186, 180,  24, 228,  24, 186, 165,  33,   5,  40, 202, 106, 114,
        213,  35, 228, 220,  27, 138,  92,  83,  32,  65, 122, 198,  87,
        171, 226, 243,  79, 248, 173,  71, 239, 123,  52, 245, 227, 193,
        191, 223, 128, 124, 190,  81,  63,   2, 143, 209, 233, 152, 128,
         19,  76, 145, 117, 194, 145,  32, 145,  96, 157,  98, 252,  23,
          5,  54,  54, 144, 215,  66,  73, 227,  46,  23, 113, 196, 184,
        139,  84, 227,  53, 106, 222, 132, 142,  28, 210,  99,  84,  53,
        202,  76, 167,  17,  71, 207, 152, 227, 161, 209, 152, 248, 134,
        142, 198, 107,  63, 240, 148, 230,  14, 196,  18,  88, 233,  86,
        218,  11, 224,  58,  34, 105, 138,  89, 156,  64, 233,  20, 109,
         72,  35, 110, 105,  67, 200, 106, 211,   7,  81, 192, 146, 160,
         16, 164,  50, 207, 103,  20, 207, 101, 178

In [32]:
# Training cell. History variable contains all the loss and accuracy at each step.

history = model.fit(context, targets, epochs = EPOCHS, verbose=VERBOSE, callbacks=callback_list)

2022-04-04 18:23:26.805219: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
1/1 [==============================] - 1s 525ms/step - loss: 179851.9531 - accuracy: 0.0000e+00
Epoch 2/1000
1/1 [==============================] - 0s 5ms/step - loss: 179818.0156 - accuracy: 0.0000e+00
Epoch 3/1000
1/1 [==============================] - 0s 5ms/step - loss: 179788.2500 - accuracy: 0.0000e+00
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 179763.0938 - accuracy: 0.0000e+00
Epoch 5/1000
1/1 [==============================] - 0s 5ms/step - loss: 179742.3438 - accuracy: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 5ms/step - loss: 179725.4688 - accuracy: 0.0000e+00
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 179711.8281 - accuracy: 0.0000e+00
Epoch 8/1000
1/1 [==============================] - 0s 5ms/step - loss: 179700.8750 - accuracy: 0.0000e+00
Epoch 9/1000
1/1 [==============================] - 0s 7ms/step - loss: 179692.0938 - accuracy: 0.0000e+00
Epoch 10/1000
1/1 [================

1/1 [==============================] - 0s 9ms/step - loss: 180133.7188 - accuracy: 0.0000e+00
Epoch 77/1000
1/1 [==============================] - 0s 13ms/step - loss: 180146.4375 - accuracy: 0.0000e+00
Epoch 78/1000
1/1 [==============================] - 0s 10ms/step - loss: 180159.3125 - accuracy: 0.0000e+00
Epoch 79/1000
1/1 [==============================] - 0s 6ms/step - loss: 180172.3594 - accuracy: 0.0000e+00
Epoch 80/1000
1/1 [==============================] - 0s 9ms/step - loss: 180185.5625 - accuracy: 0.0000e+00
Epoch 81/1000
1/1 [==============================] - 0s 8ms/step - loss: 180198.9219 - accuracy: 0.0000e+00
Epoch 82/1000
1/1 [==============================] - 0s 6ms/step - loss: 180212.4062 - accuracy: 0.0000e+00
Epoch 83/1000
1/1 [==============================] - 0s 8ms/step - loss: 180226.0938 - accuracy: 0.0000e+00
Epoch 84/1000
1/1 [==============================] - 0s 7ms/step - loss: 180239.9219 - accuracy: 0.0000e+00
Epoch 85/1000
1/1 [=====================

1/1 [==============================] - 0s 12ms/step - loss: 181529.6094 - accuracy: 0.0000e+00
Epoch 152/1000
1/1 [==============================] - 0s 9ms/step - loss: 181554.2500 - accuracy: 0.0000e+00
Epoch 153/1000
1/1 [==============================] - 0s 9ms/step - loss: 181579.0312 - accuracy: 0.0000e+00
Epoch 154/1000
1/1 [==============================] - 0s 10ms/step - loss: 181604.1094 - accuracy: 0.0000e+00
Epoch 155/1000
1/1 [==============================] - 0s 13ms/step - loss: 181629.2188 - accuracy: 0.0000e+00
Epoch 156/1000
1/1 [==============================] - 0s 10ms/step - loss: 181654.5156 - accuracy: 0.0000e+00
Epoch 157/1000
1/1 [==============================] - 0s 7ms/step - loss: 181680.0000 - accuracy: 0.0000e+00
Epoch 158/1000
1/1 [==============================] - 0s 20ms/step - loss: 181705.5938 - accuracy: 0.0000e+00
Epoch 159/1000
1/1 [==============================] - 0s 10ms/step - loss: 181731.3906 - accuracy: 0.0000e+00
Epoch 160/1000
1/1 [========

1/1 [==============================] - 0s 7ms/step - loss: 183814.7188 - accuracy: 0.0000e+00
Epoch 227/1000
1/1 [==============================] - 0s 7ms/step - loss: 183851.2812 - accuracy: 0.0000e+00
Epoch 228/1000
1/1 [==============================] - 0s 6ms/step - loss: 183887.7344 - accuracy: 0.0000e+00
Epoch 229/1000
1/1 [==============================] - 0s 53ms/step - loss: 183924.3906 - accuracy: 0.0000e+00
Epoch 230/1000
1/1 [==============================] - 0s 16ms/step - loss: 183961.0938 - accuracy: 0.0000e+00
Epoch 231/1000
1/1 [==============================] - 0s 10ms/step - loss: 183998.0312 - accuracy: 0.0000e+00
Epoch 232/1000
1/1 [==============================] - 0s 7ms/step - loss: 184035.3125 - accuracy: 0.0000e+00
Epoch 233/1000
1/1 [==============================] - 0s 8ms/step - loss: 184072.5156 - accuracy: 0.0000e+00
Epoch 234/1000
1/1 [==============================] - 0s 8ms/step - loss: 184109.9375 - accuracy: 0.0000e+00
Epoch 235/1000
1/1 [===========

...

1/1 [==============================] - 0s 14ms/step - loss: 213149.3438 - accuracy: 0.0000e+00
Epoch 674/1000
1/1 [==============================] - 0s 18ms/step - loss: 213235.5312 - accuracy: 0.0000e+00
Epoch 675/1000
1/1 [==============================] - 0s 11ms/step - loss: 213328.0312 - accuracy: 0.0000e+00
Epoch 676/1000
1/1 [==============================] - 0s 157ms/step - loss: 213420.0625 - accuracy: 0.0000e+00
Epoch 677/1000
1/1 [==============================] - 0s 11ms/step - loss: 213512.6406 - accuracy: 0.0000e+00
Epoch 678/1000
1/1 [==============================] - 0s 11ms/step - loss: 213600.8906 - accuracy: 0.0000e+00
Epoch 679/1000
1/1 [==============================] - 0s 23ms/step - loss: 213689.1719 - accuracy: 0.0000e+00
Epoch 680/1000
1/1 [==============================] - 0s 7ms/step - loss: 213779.0156 - accuracy: 0.0000e+00
Epoch 681/1000
1/1 [==============================] - 0s 6ms/step - loss: 213870.6250 - accuracy: 0.0000e+00
Epoch 682/1000
1/1 [======

1/1 [==============================] - ETA: 0s - loss: 220045.5312 - accuracy: 0.0000e+0 - 0s 14ms/step - loss: 220045.5312 - accuracy: 0.0000e+00
Epoch 749/1000
1/1 [==============================] - 0s 38ms/step - loss: 220148.2188 - accuracy: 0.0000e+00
Epoch 750/1000
1/1 [==============================] - 0s 16ms/step - loss: 220239.4844 - accuracy: 0.0000e+00
Epoch 751/1000
1/1 [==============================] - 0s 20ms/step - loss: 220338.3750 - accuracy: 0.0000e+00
Epoch 752/1000
1/1 [==============================] - 0s 17ms/step - loss: 220424.5000 - accuracy: 0.0000e+00
Epoch 753/1000
1/1 [==============================] - 0s 18ms/step - loss: 220522.5156 - accuracy: 0.0000e+00
Epoch 754/1000
1/1 [==============================] - 0s 23ms/step - loss: 220618.8125 - accuracy: 0.0000e+00
Epoch 755/1000
1/1 [==============================] - 0s 16ms/step - loss: 220712.8281 - accuracy: 0.0000e+00
Epoch 756/1000
1/1 [==============================] - 0s 16ms/step - loss: 220808.4

1/1 [==============================] - 0s 34ms/step - loss: 227169.8438 - accuracy: 0.0000e+00
Epoch 823/1000
1/1 [==============================] - 0s 13ms/step - loss: 227274.7656 - accuracy: 0.0000e+00
Epoch 824/1000
1/1 [==============================] - 0s 40ms/step - loss: 227366.7031 - accuracy: 0.0000e+00
Epoch 825/1000
1/1 [==============================] - 0s 22ms/step - loss: 227470.1719 - accuracy: 0.0000e+00
Epoch 826/1000
1/1 [==============================] - 0s 33ms/step - loss: 227563.4375 - accuracy: 0.0000e+00
Epoch 827/1000
1/1 [==============================] - 0s 52ms/step - loss: 227661.0312 - accuracy: 0.0000e+00
Epoch 828/1000
1/1 [==============================] - 0s 20ms/step - loss: 227762.1719 - accuracy: 0.0000e+00
Epoch 829/1000
1/1 [==============================] - 0s 29ms/step - loss: 227854.6562 - accuracy: 0.0000e+00
Epoch 830/1000
1/1 [==============================] - 0s 37ms/step - loss: 227952.5469 - accuracy: 0.0000e+00
Epoch 831/1000
1/1 [=====

1/1 [==============================] - 0s 11ms/step - loss: 234517.3125 - accuracy: 0.0000e+00
Epoch 897/1000
1/1 [==============================] - 0s 32ms/step - loss: 234616.7188 - accuracy: 0.0000e+00
Epoch 898/1000
1/1 [==============================] - 0s 20ms/step - loss: 234721.7812 - accuracy: 0.0000e+00
Epoch 899/1000
1/1 [==============================] - ETA: 0s - loss: 234819.8281 - accuracy: 0.0000e+0 - 0s 37ms/step - loss: 234819.8281 - accuracy: 0.0000e+00
Epoch 900/1000
1/1 [==============================] - 0s 20ms/step - loss: 234917.0000 - accuracy: 0.0000e+00
Epoch 901/1000
1/1 [==============================] - 0s 8ms/step - loss: 235021.9688 - accuracy: 0.0000e+00
Epoch 902/1000
1/1 [==============================] - 0s 10ms/step - loss: 235118.0625 - accuracy: 0.0000e+00
Epoch 903/1000
1/1 [==============================] - 0s 8ms/step - loss: 235216.3750 - accuracy: 0.0000e+00
Epoch 904/1000
1/1 [==============================] - 0s 17ms/step - loss: 235319.281

1/1 [==============================] - 0s 154ms/step - loss: 242011.1406 - accuracy: 0.0000e+00
Epoch 971/1000
1/1 [==============================] - 0s 11ms/step - loss: 242096.3281 - accuracy: 0.0000e+00
Epoch 972/1000
1/1 [==============================] - 0s 9ms/step - loss: 242213.3125 - accuracy: 0.0000e+00
Epoch 973/1000
1/1 [==============================] - 0s 7ms/step - loss: 242324.6719 - accuracy: 0.0000e+00
Epoch 974/1000
1/1 [==============================] - 0s 10ms/step - loss: 242396.8125 - accuracy: 0.0000e+00
Epoch 975/1000
1/1 [==============================] - 0s 10ms/step - loss: 242509.4688 - accuracy: 0.0000e+00
Epoch 976/1000
1/1 [==============================] - 0s 9ms/step - loss: 242604.7969 - accuracy: 0.0000e+00
Epoch 977/1000
1/1 [==============================] - 0s 11ms/step - loss: 242716.9062 - accuracy: 0.0000e+00
Epoch 978/1000
1/1 [==============================] - 0s 9ms/step - loss: 242823.9844 - accuracy: 0.0000e+00
Epoch 979/1000
1/1 [========

In [33]:
# The variable word_embeddings contains all the embeddings

word_embeddings = embedding_layer.get_weights()[0]

In [34]:
word_embeddings[0]

array([-0.02066271, -0.04141219, -0.0393253 ,  0.01212696, -0.03306778,
        0.00560141,  0.03302142, -0.03327538], dtype=float32)

In [35]:
words = []
for line in file_details['line_words']:
    for wd in line:
        words.append(wd)

NameError: name 'file_details' is not defined

In [ ]:
file_details['padded_context'][0]

In [ ]:
words

In [ ]:
# Get the searched line and word indeces
def get_indices(word):
    line_index = 0
    word_index = 0
    for line in file_details['line_words']:
        for wrd in line:
            if wrd==word:
                return (line_index, word_index)
            word_index = word_index + 1
        line_index = line_index + 1
    return (-1,-1)

In [ ]:
# Fetch the word embeddings

def fetch_embedding(line_index, word_index, word_embeddings):
    return word_embeddings[line_index]

In [ ]:
# Function to find the least distant word with the help of word_embeddings. 
# given a word and the learned word_embeddings, this function will return the next (closest) word.
# evaluation_method can be cosine or eculedian 

def closest_word(word, word_embeddings, evaluation_method):
    line_index, word_index = get_indices(word)
    print(word_embeddings.shape)
    if (line_index == -1) or (word_index == -1):
        print("searched word not found in the given context!")
    else:
        embedding = fetch_embedding(line_index, word_index, word_embeddings)
        for embd in word_embeddings:
            result = 1 - spatial.distance.cosine(embedding, embd)
            print(result, line_index, word_index)
    return "word"

In [ ]:
closest_word("2000", word_embeddings, 'cosine')

In [ ]:
word_embeddings

In [ ]:
file_details['line_words'][0][177]

In [ ]:
file_details['padded_context'][0][0]

In [ ]:
file_details

In [ ]:
model.predict()

In [ ]:
cont = []
targ = []
for ctx in context:
    for cx in ctx:
        cont.append(cx)
        targ.append(np.argmax(pred[cx]))
df = pd.DataFrame({'Word':cont, 'Next Predicted Word':targ})

In [ ]:
df